In [4]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io
import base64


from sklearn.preprocessing import MinMaxScaler

from warnings import filterwarnings
filterwarnings('ignore')

# Loading and Understanding Data

In [6]:
df = pd.read_csv("AusApparalSales4thQrt2020.csv")

In [7]:
df.head()

,Date,Time,State,Group,Unit,Sales
0,1-Oct-2020,Morning,WA,Kids,8,20000
1,1-Oct-2020,Morning,WA,Men,8,20000
2,1-Oct-2020,Morning,WA,Women,4,10000
3,1-Oct-2020,Morning,WA,Seniors,15,37500
4,1-Oct-2020,Afternoon,WA,Kids,3,7500


In [8]:
# checking info of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    7560 non-null   object
 1   Time    7560 non-null   object
 2   State   7560 non-null   object
 3   Group   7560 non-null   object
 4   Unit    7560 non-null   int64 
 5   Sales   7560 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 354.5+ KB


# Data Wrangling

In [9]:
# Fucntion to check missing values
def check_missing_values(data):
    """
    Check for missing values in the dataset and return a summary.
    """
    missing_summary = data.isna().sum()
    print("Missing Values Summary:")
    return missing_summary

In [10]:
check_missing_values(df)

Missing Values Summary:


Date     0
Time     0
State    0
Group    0
Unit     0
Sales    0
dtype: int64

We can clearly see that there are no missing values in the dataset

### b. Normalizing the Data

In [11]:
def normalize_data(data, columns_to_normalize):
    """
    Normalize the specified columns of the dataset using Min-Max Scaling.
    
    Args:
        data (pd.DataFrame): The input DataFrame.
        columns_to_normalize (list): List of column names to normalize.
    
    Returns:
        pd.DataFrame: DataFrame with normalized columns.
    """
    scaler = MinMaxScaler()
    data_normalized = data.copy()
    
    # Apply Min-Max Scaling to the selected columns
    data_normalized[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
    
    print("\nNormalized Data Sample:")
    print(data_normalized.head())
    
    return data_normalized

In [12]:
columns_to_normalize = ["Unit", "Sales"]  # Specifying columns to normalize
data_normalized = normalize_data(df, columns_to_normalize)


Normalized Data Sample:
         Date        Time State     Group      Unit     Sales
0  1-Oct-2020     Morning    WA      Kids  0.095238  0.095238
1  1-Oct-2020     Morning    WA       Men  0.095238  0.095238
2  1-Oct-2020     Morning    WA     Women  0.031746  0.031746
3  1-Oct-2020     Morning    WA   Seniors  0.206349  0.206349
4  1-Oct-2020   Afternoon    WA      Kids  0.015873  0.015873


# Data Analysis

### a. Description Statistical summary of the data

In [13]:
data_normalized.describe().T

,count,mean,std,min,25%,50%,75%,max
Unit,7560.0,0.254054,0.204784,0.0,0.095238,0.190476,0.380952,1.0
Sales,7560.0,0.254054,0.204784,0.0,0.095238,0.190476,0.380952,1.0


In [14]:
data_normalized.describe(include = 'object').T

,count,unique,top,freq
Date,7560,90,1-Oct-2020,84
Time,7560,3,Morning,2520
State,7560,7,WA,1080
Group,7560,4,Kids,1890


### Data Insights and Explanation

#### Numerical Data Summary
- **Count**: Both `Unit` and `Sales` have 7560 entries, meaning there are no missing values in these columns.
- **Mean and Distribution**: The average (`mean`) value for both `Unit` and `Sales` is `0.254`. With a standard deviation of `0.2048`, the data is moderately spread around the mean. The values range from `0.0` to `1.0`, suggesting that the data might be normalized or scaled.
- **Quartiles**: Most of the data lies between `0.095` (25th percentile) and `0.381` (75th percentile), showing a right-skewed distribution with lower values being more frequent.

---

#### Categorical Data Summary
- **Date**: The dataset spans 90 unique dates, with `1-Oct-2020` appearing most frequently (84 times). This might indicate a significant event or peak activity on this date.
- **Time**: There are three time categories, with `Morning` being the most common (2520 entries). This suggests that most activities occurred during the morning hours.
- **State**: Seven states are represented, with `WA` being the most frequent (1080 entries). This could indicate that the dataset is focused on or has a bias toward activity in Washington.
- **Group**: Four groups are present, and `Kids` is the largest category (1890 entries), suggesting a focus or higher participation from the "Kids" group.

---

#### Insights
1. **Numerical Data**: The similarity in the statistics for `Unit` and `Sales` implies a potential relationship between the two columns. The distribution indicates that lower values are more common, with occasional higher values pushing the max to `1.0`.
2. **Categorical Data**:
   - The dominance of `Morning`, `WA`, and the `Kids` group might highlight specific trends or patterns in the dataset.
   - The frequent occurrence of `1-Oct-2020` might require further investigation to understand what caused this spike.


### Identify highest sales group by lowest

In [15]:
def identify_highest_lowest_sales_group(data):
    """
    Identify the group with the highest and lowest total sales.
    
    Args:
        data (pd.DataFrame): The input DataFrame.
    
    Returns:
        tuple: Group with highest sales, Group with lowest sales
    """
    # Group by 'Group' and sum the 'Sales' for each group
    sales_by_group = df.groupby('Group')['Sales'].sum()
    
    # Identify the group with the highest and lowest sales
    highest_sales_group = sales_by_group.idxmax()
    lowest_sales_group = sales_by_group.idxmin()
    
    highest_sales_value = sales_by_group.max()
    lowest_sales_value = sales_by_group.min()
    
    print(f"Group with the Highest Sales: {highest_sales_group} ({highest_sales_value})")
    print(f"Group with the Lowest Sales: {lowest_sales_group} ({lowest_sales_value})")
    
    return highest_sales_group, lowest_sales_group


identify_highest_lowest_sales_group(normalize_data)

Group with the Highest Sales:  Men (85750000)
Group with the Lowest Sales:  Seniors (84037500)


(' Men', ' Seniors')

#### Group Sales Insights

- **Men**: The group with the highest sales, generating a total of 85,750,000 in sales. This indicates a strong demand for men's clothing.
- **Seniors**: The group with the lowest sales, totaling 84,037,500. This suggests that sales in this demographic are comparatively lower, possibly due to lower demand or limited product offerings for seniors.


### c. Generating Weekly, Monthly, and Quarterly Reports

In [16]:
data_normalized.head()

,Date,Time,State,Group,Unit,Sales
0,1-Oct-2020,Morning,WA,Kids,0.095238,0.095238
1,1-Oct-2020,Morning,WA,Men,0.095238,0.095238
2,1-Oct-2020,Morning,WA,Women,0.031746,0.031746
3,1-Oct-2020,Morning,WA,Seniors,0.206349,0.206349
4,1-Oct-2020,Afternoon,WA,Kids,0.015873,0.015873


In [17]:
def generate_time_period_reports(data):
    """
    Generate weekly, monthly, and quarterly reports based on the sales data.
    
    Args:
        data (pd.DataFrame): The input DataFrame.
    
    Returns:
        tuple: Weekly, Monthly, and Quarterly reports DataFrames.
    """
    # Converting 'Date' column to datetime format
    data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%Y')
    
    # Setting the 'Date' column as the index for easier resampling
    data.set_index('Date', inplace=True)
    
    # Weekly report: Aggregating sales by week
    weekly_report = data.resample('W')['Sales'].sum()
    
    # Monthly report: Aggregating sales by month
    monthly_report = data.resample('M')['Sales'].sum()
    
    # Quarterly report: Aggregating sales by quarter
    quarterly_report = data.resample('Q')['Sales'].sum()
    
    # Printing the reports
    print("\nWeekly Sales Report:")
    print(weekly_report)
    
    print("\nMonthly Sales Report:")
    print(monthly_report)
    
    print("\nQuarterly Sales Report:")
    print(quarterly_report)
    
    return weekly_report, monthly_report, quarterly_report

weekly_report, monthly_report, quarterly_report = generate_time_period_reports(data_normalized)


Weekly Sales Report:
Date
2020-10-04     84.857143
2020-10-11    152.777778
2020-10-18    150.476190
2020-10-25    151.587302
2020-11-01    122.460317
2020-11-08    113.809524
2020-11-15    115.761905
2020-11-22    115.380952
2020-11-29    117.698413
2020-12-06    169.412698
2020-12-13    181.492063
2020-12-20    182.317460
2020-12-27    183.047619
2021-01-03     79.571429
Freq: W-SUN, Name: Sales, dtype: float64

Monthly Sales Report:
Date
2020-10-31    645.650794
2020-11-30    495.761905
2020-12-31    779.238095
Freq: M, Name: Sales, dtype: float64

Quarterly Sales Report:
Date
2020-12-31    1920.650794
Freq: Q-DEC, Name: Sales, dtype: float64


#### Weekly Sales Report

The weekly sales report shows significant fluctuations in sales:
- **Highest sales**: Last week of December (2020-12-27) at **31.77 million**, reflecting a surge in year-end shopping.
- **Lowest sales**: First week of January (2021-01-03) at **13.79 million**, possibly due to a post-holiday slowdown.

#### Monthly Sales Report

The monthly sales report indicates a steady decline in sales over three months:
- **October**: Highest sales at **114.29 million**.
- **November**: **90.68 million**.
- **December**: **135.33 million**, suggesting a drop in sales before the holiday season peaked.

#### Quarterly Sales Report

The quarterly report shows:
- **Total sales in Q4 2020**: **340.3 million**.
- Highlights the strong sales performance during the holiday season, contributing to a substantial portion of the year’s revenue.

#### Insights

These trends suggest:
- **Targeted promotions**: In low-performing months.
- **Emphasis**: On high sales weeks leading up to the holidays.


# Data Visualization

In [18]:
# Initializing Dash app
app = dash.Dash(__name__)

# 1. State-wise sales analysis for different demographic groups
state_group_sales = data_normalized.groupby(['State', 'Group'])['Sales'].sum().reset_index()
fig1 = px.bar(state_group_sales, x='State', y='Sales', color='Group', 
              title="State-wise Sales Analysis for Different Demographic Groups", 
              labels={"Sales": "Total Sales", "State": "State", "Group": "Demographic Group"})

# 2. Group-wise sales analysis across various states
group_state_sales = data_normalized.groupby(['Group', 'State'])['Sales'].sum().reset_index()
fig2 = px.bar(group_state_sales, x='Group', y='Sales', color='State', 
              title="Group-wise Sales Analysis Across States", 
              labels={"Sales": "Total Sales", "Group": "Demographic Group", "State": "State"})

# 3. Time-of-the-day analysis: peak and off-peak sales periods
time_of_day_sales = data_normalized.groupby('Time')['Sales'].sum().reset_index()
fig3 = px.line(time_of_day_sales, x='Time', y='Sales', 
               title="Time-of-the-Day Sales Analysis", 
               labels={"Sales": "Total Sales", "Time": "Time of the Day"})

charts = {}

# Daily Sales Trend (Seaborn plot converted to Plotly)
daily_sales = data_normalized['Sales'].resample('D').sum()

plt.figure(figsize=(10, 6))
sns.lineplot(x=daily_sales.index, y=daily_sales.values, label='Daily Sales', color='blue')
plt.title('Daily Sales Trend', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sales', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
img = io.BytesIO()
plt.savefig(img, format='png')
plt.close()
img.seek(0)
charts['daily'] = base64.b64encode(img.getvalue()).decode('utf-8')

# Weekly Sales Trend (Seaborn plot converted to Plotly)
weekly_sales = data_normalized['Sales'].resample('W').sum()

plt.figure(figsize=(10, 6))
sns.lineplot(x=weekly_sales.index, y=weekly_sales.values, label='Weekly Sales', color='green')
plt.title('Weekly Sales Trend', fontsize=16)
plt.xlabel('Week', fontsize=12)
plt.ylabel('Sales', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
img = io.BytesIO()
plt.savefig(img, format='png')
plt.close()
img.seek(0)
charts['weekly'] = base64.b64encode(img.getvalue()).decode('utf-8')

# Monthly Sales Trend (Seaborn plot converted to Plotly)
monthly_sales = data_normalized['Sales'].resample('M').sum()

plt.figure(figsize=(10, 6))
sns.lineplot(x=monthly_sales.index, y=monthly_sales.values, label='Monthly Sales', color='orange')
plt.title('Monthly Sales Trend', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Sales', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
img = io.BytesIO()
plt.savefig(img, format='png')
plt.close()
img.seek(0)
charts['monthly'] = base64.b64encode(img.getvalue()).decode('utf-8')

# Layout of the Dash app
app.layout = html.Div([
    html.H1("Sales and Marketing Dashboard"),
    html.Div([
        html.H3("State-wise Sales Analysis for Different Demographic Groups"),
        dcc.Graph(figure=fig1)
    ]),
    html.Div([
        html.H3("Group-wise Sales Analysis Across States"),
        dcc.Graph(figure=fig2)
    ]),
    html.Div([
        html.H3("Time-of-the-Day Sales Analysis"),
        dcc.Graph(figure=fig3)
    ]),
    html.Div([
        html.H3("Daily Sales Trend"),
        html.Img(src=f"data:image/png;base64,{charts['daily']}")
    ]),
    html.Div([
        html.H3("Weekly Sales Trend"),
        html.Img(src=f"data:image/png;base64,{charts['weekly']}")
    ]),
    html.Div([
        html.H3("Monthly Sales Trend"),
        html.Img(src=f"data:image/png;base64,{charts['monthly']}")
    ])
])

# Running the app
if __name__ == '__main__':
    app.run_server(debug=True)

### Final Analysis Report with Recommendations

#### 1. **State-wise Sales Analysis for Different Demographic Groups (Kids, Men, Women, Seniors)**

**Graph Description:**  
This bar chart provides a state-wise breakdown of total sales across various demographic groups: Kids, Men, Women, and Seniors. It shows the contribution of each demographic group to the total sales in different states such as Victoria (VIC), New South Wales (NSW), Northern Territory (NT), Tasmania (TAS), Queensland (QLD), South Australia (SA), and Western Australia (WA).

**Key Insights:**
- **Victoria (VIC)** leads in total sales across all demographics, followed by **New South Wales (NSW)**.
- **Northern Territory (NT)** and **Tasmania (TAS)** have the lowest total sales figures.
- **Kids** and **Men** are the dominant demographic groups contributing to sales, while **Seniors** and **Women** have relatively smaller contributions across all states.
- States like **VIC** and **NSW** contribute significantly to sales, while **NT** and **TAS** show weaker performance, indicating a potential area for sales growth.

**Recommendations:**
- **Focus on Growth in NT and TAS:** Given the lower sales in NT and TAS, targeted sales programs focusing on these regions could help boost revenue. Specific promotions or localized marketing strategies might increase engagement with these regions.
- **Boost Female and Senior Sales:** While Men and Kids are strong performers, there is potential to increase the contribution of Women and Seniors. Special offers or tailored product lines for these groups could help increase sales.

#### 2. **Group-wise Sales Analysis Across Various States (Kids, Men, Seniors, Women)**

**Graph Description:**  
This grouped bar chart shows sales by demographic groups (Kids, Men, Women, Seniors) across different states. The height of each bar represents the total sales of each demographic, while the color within each bar represents the contribution of different states.

**Key Insights:**
- **Kids** and **Men** consistently show the highest sales figures across all states, with **VIC** contributing the largest share in every group.
- **NT** and **TAS** contribute the least across all groups, confirming the trend observed in the state-wise analysis.
- The **Seniors** and **Women** groups, while showing moderate sales, could benefit from targeted strategies to increase engagement.

**Recommendations:**
- **State-Specific Strategies for Demographic Groups:** Given that VIC contributes the most in every demographic group, more localized campaigns tailored to specific states, especially NT and TAS, could potentially improve the performance of those regions.
- **Explore Product Expansion for Women and Seniors:** Launching targeted campaigns or product lines aimed at **Women** and **Seniors** could help tap into these underperforming demographic groups.

#### 3. **Time-of-the-Day Sales Analysis (Morning, Afternoon, Evening)**

**Graph Description:**  
This line plot illustrates the total sales distribution over different times of the day: Morning, Afternoon, and Evening. It shows that sales peak in the **Morning** and **Afternoon**, with a noticeable dip during the **Evening**.

**Key Insights:**
- The **Morning** and **Afternoon** are the peak sales periods, while the **Evening** sees a noticeable drop in sales.
- This pattern indicates that customer activity tends to decrease in the evening, possibly due to other factors like time of day or availability.

**Recommendations:**
- **Leverage Peak Sales Hours for Promotions:** Since the Morning and Afternoon have higher sales, sales campaigns or promotions should be concentrated during these periods to maximize revenue.
- **Evening Promotions or Offers:** Consider introducing targeted promotions in the Evening to try and stimulate sales, such as discounts or limited-time offers.

#### 4. **Daily Sales Trend (October 2020 to January 2021)**

**Graph Description:**  
This line plot displays the daily sales trend, showing fluctuations over time. Initially, sales are relatively stable around 4 million but experience a sharp drop in early November. After the drop, sales stabilize around 3.5 million and show a strong upward trend in early December.

**Key Insights:**
- **Sharp Drop in November:** The drop in early November could indicate an anomaly or a seasonal dip in sales.
- **Recovery in December:** Sales recover towards the end of November and peak in early December, indicating a rebound or the impact of holiday-related sales.

**Recommendations:**
- **Analyze November Dip:** Investigate the cause of the November sales dip. Was it due to market conditions, external events, or internal factors? Understanding this could help avoid future dips.
- **Capitalize on the December Surge:** Given the sharp recovery in December, capitalizing on this trend with strategic promotions or stock management can further boost sales during this period.

#### 5. **Weekly Sales Trend (October 2020 to January 2021)**

**Graph Description:**  
The weekly sales trend shows fluctuations with sales peaking at over 2.5 million by mid-October. A slight decline follows in early November, with a surge in December and a sharp drop at the end of the period.

**Key Insights:**
- **Sales Surge in December:** December shows the highest peak in weekly sales, aligning with typical holiday season peaks.
- **Fluctuations in October and November:** Sales stabilize and dip between October and November, showing less consistency before the year-end surge.

**Recommendations:**
- **Prepare for Seasonal Peaks:** Use insights from December's sales surge to plan for future holiday seasons, ensuring sufficient stock and targeted marketing.
- **Manage Early November Slump:** Strategies like promotions or product launches could help counteract the decline in early November, smoothing the sales curve.

#### 6. **Monthly Sales Trend (October 2020 to January 2021)**

**Graph Description:**  
This monthly trend graph shows sales starting at around 1.1 billion in early November, dipping to about 900 million by the end of November, and then steadily rising to 1.35 billion by the end of December.

**Key Insights:**
- **November Dip:** A noticeable dip in November is followed by a steady rise in December, peaking at 1.35 billion.
- **Strong Recovery in December:** The recovery and growth in December indicate that the company successfully capitalized on the end-of-year sales period.

**Recommendations:**
- **Boost November Sales:** Address the November dip with special offers, early holiday promotions, or campaigns targeting key demographics.
- **Capitalize on December:** The end-of-year period can be a major revenue generator. Plan early to optimize stock, pricing, and marketing efforts during this peak time.

---

#### **Conclusion & Final Recommendations:**

- **State-Specific Growth Initiatives:** Focus on boosting sales in underperforming states (NT and TAS) through localized campaigns or product introductions aimed at specific demographic groups.
- **Tailored Demographic Campaigns:** Women's and Seniors' segments show potential for growth. Develop tailored offerings and promotions to increase sales in these underperforming groups.
- **Optimize Sales Periods:** Use the peak sales times (Morning and Afternoon) to maximize revenue through promotions. Consider strategies for evening sales to reduce the current dip.
- **Seasonal Planning:** Prepare for seasonal trends, particularly focusing on December’s sales surge while managing the November dip. Anticipate similar trends for future years and plan marketing strategies accordingly.
